In [1]:
from llama_cpp import Llama
import chromadb
import warnings
warnings.filterwarnings('ignore')


# onderstaande koste mij ongeveer een halfuur
# pip install llama_cpp-python


In [2]:
documents = []
metadata = []
ids = []
counter = 0
with open('data/Opleidngsdeel_OER_HBO-ICT_Zwolle_2024-2025_Filterd_Modified_Long_Lines.txt', 'r', encoding='utf-8') as lines:
    for line in lines:
        line=line.split(" - ") #spells are separated by their description by a " - "
        metadata.append({'inhoud': line[1]}) #use the spell name as metadata. ChromaDB requires a dictionary here
        documents.append(line[0]) #use the description as documents
        ids.append(str(counter)) #we also need an ID so use the spell name again.
        counter+=1
print(ids)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '15

In [3]:
client = chromadb.Client()

# create or open collection named 'spells' 
collection = client.get_or_create_collection("information")

# add spells to the collection, ignored for ids that already exist 
collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

In [4]:
llm = Llama.from_pretrained(
	repo_id="BramVanroy/GEITje-7B-ultra-GGUF",
	filename="geitje-7b-ultra-q8_0.gguf",
)

llm.create_chat_completion(
	messages = [
		{
			"role": "user",
			"content": "Wat is de hoofdstad van Frankrijk?"
		}
	]
)

llama_model_loader: loaded meta data with 26 key-value pairs and 291 tensors from C:\Users\raymo\.cache\huggingface\hub\models--BramVanroy--GEITje-7B-ultra-GGUF\snapshots\00da1c06460774d6982d608e495dac102ead3ec1\.\geitje-7b-ultra-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = geitje-7b-ultra
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:            

{'id': 'chatcmpl-d09a1007-3823-4d87-a76c-df30ece92c4a',
 'object': 'chat.completion',
 'created': 1729073942,
 'model': 'C:\\Users\\raymo\\.cache\\huggingface\\hub\\models--BramVanroy--GEITje-7B-ultra-GGUF\\snapshots\\00da1c06460774d6982d608e495dac102ead3ec1\\.\\geitje-7b-ultra-q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'De hoofdstad van Frankrijk is Parijs. Parijs staat bekend als de "Lichtstad" en is een van de meest iconische steden ter wereld. Het is niet alleen de hoofdstad van Frankrijk, maar ook een cultureel, economisch en politiek centrum van het land. Parijs huisvest vele bezienswaardigheden, zoals de Eiffeltoren, de Notre-Dame en het Louvre, evenals tal van kunstgalerijen, parken en historische plekken.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 27, 'completion_tokens': 125, 'total_tokens': 152}}

In [10]:
user_input = input("Wat is jouw vraag?:")
results = collection.query(
    query_texts=[user_input],
    n_results=1
)
results

{'ids': [['819']],
 'distances': [[1.0952637195587158]],
 'metadatas': [[{'inhoud': 'onderwijs- en examenregeling van de opleiding.\n'}]],
 'embeddings': None,
 'documents': [['BIJLAGEN BIJ ONDERWIJS- EN EXAMENREGELING']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [11]:
result=results['metadatas'][0][0]['inhoud']+" - " + results['documents'][0][0][:-1]
result

'onderwijs- en examenregeling van de opleiding.\n - BIJLAGEN BIJ ONDERWIJS- EN EXAMENREGELIN'

In [12]:
prompt = f"""
voorgestelde antwoord: {result}
User input: {user_input}
stel een nieuw antwoord voor op basis van het voorgestelde antwoord.
"""
print("Prompt: ", prompt)
inference = llm(prompt, max_tokens=100)
generated_text = inference['choices'][0]['text']
print("Inference: ", generated_text)

Prompt:  
voorgestelde antwoord: onderwijs- en examenregeling van de opleiding.
 - BIJLAGEN BIJ ONDERWIJS- EN EXAMENREGELIN
User input: wat is bim?
stel een nieuw antwoord voor op basis van het voorgestelde antwoord.



Llama.generate: 11 prefix-match hit, remaining 71 prompt tokens to eval
llama_perf_context_print:        load time =    6641.65 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    71 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    39 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   16651.40 ms /   110 tokens


Inference:  voorgestelde antwoord: bim staat voor Building Information Modeling, een methode om gebouwen en infrastructuur te modelleren in digitale vorm.
